In [35]:
import numpy as np
import pandas as pd
import biogeme
import sklearn
from lightgbm import LGBMClassifier

In [10]:
ned_data = pd.read_table("Data/netherlandsRP.dat")

In [16]:
ned_data.columns

Index(['id', 'rp', 'sp', 'choice', 'purpose', 'npersons', 'age',
       'employ_status', 'mainearn', 'arrival_time', 'gender', 'rail_ivtt',
       'rail_cost', 'rail_transfers', 'rail_acc_time', 'rail_egr_time',
       'rail_acc_mode', 'rail_egr_mode', 'seat_status', 'car_ivtt', 'car_cost',
       'car_walk_time', 'car_parking_fee', 'rail_comfort', 'rp_transfer',
       'rp_choice', 'rp_rail_ovt', 'rp_car_ovt'],
      dtype='object')

228

In [28]:
split = 0.7
mask = np.random.random(size=len(ned_data))

In [30]:
X = ned_data[['rail_cost', 'car_cost']]

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.20, random_state=42)

In [50]:
params = {
          "monotone_constraints": [1, 1]
}
lightGBM = LGBMClassifier()
#lightGBM.set_params(params)

In [51]:
lightGBM.fit(X_train, y_train)

LGBMClassifier()

In [52]:
y_pred=lightGBM.predict(X_validate)

In [53]:
from sklearn.metrics import accuracy_score

In [55]:
accuracy_score(y_validate, y_pred)

0.5869565217391305

In [57]:
lightGBM.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [59]:
lightGBM.feature_importances_

array([279, 296])

61

In [90]:
import biogeme.database as db
train_data = X_train.join(y_train)
database = db.Database("ned_data", train_data)
globals().update(database.variables)

In [91]:
from biogeme.expressions import Beta

ASC_c = Beta('ASC_c',0,None,None,0)
ASC_pt = Beta('ASC_pt', 0,None,None,1)

B_COST_c = Beta('B_COST_c', 0, None, None, 0)
B_COST_pt = Beta('B_COST_pt', 0, None, None, 0)

In [92]:
V_c = ASC_c + B_COST_c * car_cost
V_pt = ASC_pt + B_COST_pt * rail_cost
V = {0: V_c, 1: V_pt}

In [93]:
avail = {0: 1, 1: 1}

In [94]:
import biogeme.models as models

import biogeme.messaging as msg
import biogeme.biogeme as bio

# observation to the log likelihood function.
logprob = models.loglogit(V, avail, rp_choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = '01logit'

# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(avail)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

              Value   Std err    t-test   p-value  Rob. Std err  Rob. t-test  \
ASC_c      0.401589  0.674769  0.595150  0.551743      0.727676     0.551878   
B_COST_c  -0.062916  0.013096 -4.804148  0.000002      0.013995    -4.495493   
B_COST_pt -0.039362  0.019114 -2.059353  0.039460      0.020146    -1.953829   

           Rob. p-value  
ASC_c          0.581032  
B_COST_c       0.000007  
B_COST_pt      0.050721  


Log likelihood for 46 validation data: -27.23098013351334
Log likelihood for 46 validation data: -26.855177449434656
Log likelihood for 46 validation data: -31.739714362438058
Log likelihood for 45 validation data: -22.21358457391922
Log likelihood for 45 validation data: -23.41357073248124
